# docTTTTTquery MSMARCO

In [1]:
# !pip install transformers --quiet
# !pip install torch --quiet
# !pip install pytorch-lightning --quiet
# !pip install pyDrive --quiet

In [43]:
triples_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\train_triples_small\\triples.train.small.tsv'
chuncks_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\train_triples_small\\triples_smalls_chunks'
model_checkpoint_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Vetores Densos\\checkpoint docTTTTTquery\\model.ckpt-1004000'

In [2]:
#Bibliotecas Padrão
import os
import random
from typing import Dict
from typing import List
from typing import Tuple
import re
import gzip
import csv
import zipfile
    
#Bibliotecas Data Science
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

#Bibliotecas Pytorch
import torch
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils.data import DataLoader
import torch.nn.functional as F

#Bibliotecas transfromers
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer

# Hardware
from multiprocessing import cpu_count


In [3]:
# Device
dev = "cuda" if torch.cuda.is_available() else "cpu"
print(dev)
device = torch.device(dev)

cpu


## Trabalhando com arquivo csv 

Funções auxiliares

In [4]:
def split_csv(path, delimiter="\t", row_limit=30000, 
    output_name_template='triples_smalls_chunk_%s.csv', output_path='.', keep_headers=True,only_first=True):
    """
    Splits a CSV file into multiple pieces.
    
    A quick bastardization of the Python CSV library.
    Arguments:
        `row_limit`: The number of rows you want in each output file
        `output_name_template`: A %s-style template for the numbered output files.
        `output_path`: Where to stick the output files
        `keep_headers`: Whether or not to print the headers in each output file.
    Example usage:
    
        >> from toolbox import csv_splitter;
        >> csv_splitter.split(csv.splitter(open('/home/ben/Desktop/lasd/2009-01-02 [00.00.00].csv', 'r')));
    
    """
    tsv_file  = open(path,encoding="cp1252",mode = 'r', errors='ignore')
    reader = csv.reader(tsv_file, delimiter="\t")

    current_piece = 0
    current_out_path = os.path.join( output_name_template  % current_piece)
    current_out_path = os.path.join( output_path,current_out_path)
    
    current_limit = row_limit

    rowlist = []

    for i, row in enumerate(reader):
        if (i + 1) > current_limit:
            current_piece += 1
            current_limit = row_limit * (current_piece+1)
            current_out_path = os.path.join( output_name_template  % current_piece )
            current_out_path = os.path.join( output_path,current_out_path)
            print(current_out_path)
            with open(current_out_path, 'w', newline='') as file:
                writer = csv.writer(file, quoting=csv.QUOTE_NONNUMERIC, delimiter='\t', quotechar='"')
                writer.writerows(rowlist)
            rowlist = []
            if (only_first):  
                break
                

        row_elems = [elem.replace("Ã¢Â€Â™","").replace("Ã¢Â€Âœ","") for elem in row]
        rowlist.append(row_elems)

In [5]:
def load_text_tsv_chunck(path,file):
    complete_path = os.path.join(path,file)
    csv_file =  open(complete_path, encoding="cp1252",mode = 'r', errors='ignore')
    csv_reader = csv.reader(csv_file, delimiter='\t')
    rows = []
    for row in csv_reader:
        rows.append(row)
    return rows

### Caso queira trabalhar com uma porção de tsv(arquivos menores)




Checando a extensão do encoding

In [6]:
with open(triples_path) as f:
    print(f)

<_io.TextIOWrapper name='C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\MS MARCO 2020\\train_triples_small\\triples.train.small.tsv' mode='r' encoding='cp1252'>


Separação dos dados

In [7]:
split_csv(triples_path, delimiter="\t", row_limit=30000,  output_name_template='triples_smalls_chunk_%s.csv', output_path=chuncks_path, keep_headers=True,only_first=True)

C:\Users\Matheus\Desktop\Pós Graduação\NLP\Projeto\Datasets\MS MARCO 2020\train_triples_small\triples_smalls_chunks\triples_smalls_chunk_1.csv


**Escolher o CHUNCK 2**

In [7]:
file = "triples_smalls_chunk_2.csv"    
ds = load_text_tsv_chunck(chuncks_path,file)
print(ds[4])

['how many calories a day are lost breastfeeding', 'Not only is breastfeeding better for the baby, however, research also says its better for the mother. Breastfeeding burns an average of 500 calories a day, with the typical range from 200 to 600 calories burned a day. Its estimated that the production of 1 oz. of breast milk burns 20 calories. The amount of calories burned depending on how much the baby eats. Breastfeeding twins burn twice as much as feeding only one baby. With twins their mom burns 1000 calories a day. Burning an extra 500 calories a day will result in one pound of weekly weight loss.', 'However, you still need some niacin each day; men need about 16 mg per day and women need 14 mg per day unless they are pregnant or nursing (pregnant and breastfeeding women have higher niacin requirements).']


### Caso queria trabalhar com os dados completos

Extração dos dados

In [16]:
# ds = load_text_untar(triples_path)
# print(ds[:2])

Printando 3 amostras

In [18]:
# for set_name, x in [('treino', ds_train)]:
#     print(f'\n{len(x)} amostras de {set_name}')
#     print(f'3 primeiras amostras {set_name}:')
#     for i, (source, target) in enumerate(x[:3]):
#         print(f'{i}: source: {source}\n   target: {target}')

## Recuperando treinamento do docTTTTquery

In [8]:
import torch
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration


tokenizer = T5Tokenizer.from_pretrained('t5-base')
config = T5Config.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained(
model_checkpoint_path, from_tf=True, config=config)
model = model.to(device)


Testando decoder top K (não esquecer end-of-sequence token </s>):

In [9]:
doc_text = 'The meaning of life as we perceive it is derived from philosophical and religious contemplation of, and scientific inquiries about existence, social ties, consciousness, and happiness. Many other issues are also involved, such as symbolic meaning, ontology, value, purpose, ethics, good and evil, free will, the existence of one or multiple gods, conceptions of God, the soul, and the afterlife. Scientific contributions focus primarily on describing related empirical facts about the universe, exploring the context and parameters concerning the "how" of life. Science also studies and can provide recommendations for the pursuit of well-being and a related conception of morality. </s>'

input_ids = tokenizer.encode(doc_text, return_tensors='pt').to(device)
outputs = model.generate(input_ids=input_ids,max_length=64,do_sample=True,top_k=10,num_return_sequences=10)

for i in range(10):
    print(f'sample {i + 1}: {tokenizer.decode(outputs[i], skip_special_tokens=True)}')

sample 1: what is the scientific definition of life and its meaning
sample 2: what is the meaning of life
sample 3: scientific definitions definition
sample 4: what scientific contribution is made to understanding the meaning of life?
sample 5: what makes up the scientific study of life?
sample 6: how is life defined in science?
sample 7: what is the scientific definition of life
sample 8: which science is concerned with understanding the meaning of life?
sample 9: what is life and its meaning quizlet
sample 10: which is related to the scientific inquiry of life?


## Trabalhando com Dataframes

In [10]:
def passage2query(passage):
    passage_input = tokenizer.encode(f"{passage} </s>", return_tensors='pt').to(device)
    passage_outputs = model.generate(input_ids=passage_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)

    return passage_outputs
    

In [11]:
def passage2queryRow(row):
#     df_queries= df_queries.apply(lambda row : passage2queryRow(row), axis = 1)
    positive_passage = row[2]
    negative_passage = row[3]
    
    positive_input = tokenizer.encode(f"{positive_passage} </s>", return_tensors='pt').to(device)
    positive_outputs = model.generate(input_ids=positive_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
    row[2] = tokenizer.decode(positive_outputs[0], skip_special_tokens=True)
    
    
    negative_input = tokenizer.encode(f"{negative_passage} </s>", return_tensors='pt').to(device)
    negative_outputs = model.generate(input_ids=negative_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
    row[3] = tokenizer.decode(negative_outputs[0], skip_special_tokens=True)
    
    return row

In [12]:
def passage2queryGPU(positive_passage,negative_passage,positive_questions,negative_questions,kwarg1):

    
    positive_input = tokenizer.encode(f"{positive_passage} </s>", return_tensors='pt').to(device)
    positive_outputs = model.generate(input_ids=positive_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
    positive_questions = tokenizer.decode(positive_outputs[0], skip_special_tokens=True)
    
    
    negative_input = tokenizer.encode(f"{negative_passage} </s>", return_tensors='pt').to(device)
    negative_outputs = model.generate(input_ids=negative_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
    negative_questions = tokenizer.decode(negative_outputs[0], skip_special_tokens=True)

In [13]:
def passage2queryDict(dict_org):
    for i in range(0,len(dict_org['ID'])):
        positive_passage = dict_org['POSITIVE_PASSAGE'][i]
        negative_passage = dict_org['NEGATIVE_PASSAGE'][i]

        positive_input = tokenizer.encode(f"{positive_passage} </s>", return_tensors='pt').to(device)
        positive_outputs = model.generate(input_ids=positive_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
        dict_org['POSITIVE_PASSAGE'][i] = tokenizer.decode(positive_outputs[0], skip_special_tokens=True)


        negative_input = tokenizer.encode(f"{negative_passage} </s>", return_tensors='pt').to(device)
        negative_outputs = model.generate(input_ids=negative_input,max_length=64,do_sample=True,top_k=1,num_return_sequences=1)
        dict_org['NEGATIVE_PASSAGE'][i] = tokenizer.decode(negative_outputs[0], skip_special_tokens=True) 
    return dict_org

In [60]:
def passage2queryDictMultiple(dict_org):
    for i in range(0,len(dict_org['ID'])):
        positive_passage = dict_org['POSITIVE_PASSAGE'][i]
        negative_passage = dict_org['NEGATIVE_PASSAGE'][i]

        positive_input = tokenizer.encode(f"{positive_passage} </s>", return_tensors='pt').to(device)
        positive_outputs = model.generate(input_ids=positive_input,max_length=64,do_sample=True,top_k=5,num_return_sequences=5)
        for j , po in enumerate(positive_outputs):
            dict_org['POSITIVE_QUERY_'+str(j+1)][i] = tokenizer.decode(po, skip_special_tokens=True)


        negative_input = tokenizer.encode(f"{negative_passage} </s>", return_tensors='pt').to(device)
        negative_outputs = model.generate(input_ids=negative_input,max_length=64,do_sample=True,top_k=5,num_return_sequences=5)
        for j , no in enumerate(negative_outputs):
            dict_org['NEGATIVE_QUERY_'+str(j+1)][i] = tokenizer.decode(no, skip_special_tokens=True)
    
    return dict_org

In [ ]:
def passage2queryMultipleAppend(ds):
    df = pd.dataframe(ds,columns = ['ID','ORIGINAL_QUERY','POSITIVE_PASSAGE','NEGATIVE_PASSAGE','POSITIVE_QUERY_1','POSITIVE_QUERY_2','POSITIVE_QUERY_3','POSITIVE_QUERY_4','POSITIVE_QUERY_5','NEGATIVE_QUERY_1','NEGATIVE_QUERY_2','NEGATIVE_QUERY_3','NEGATIVE_QUERY_4','NEGATIVE_QUERY_5'] )
    for i,row in enumerate(ds):
        positive_passage = row[2]
        negative_passage = dict_org['NEGATIVE_PASSAGE'][i]

        positive_input = tokenizer.encode(f"{positive_passage} </s>", return_tensors='pt').to(device)
        positive_outputs = model.generate(input_ids=positive_input,max_length=64,do_sample=True,top_k=5,num_return_sequences=5)
        for j , po in enumerate(positive_outputs):
            dict_org['POSITIVE_QUERY_'+str(j+1)][i] = tokenizer.decode(po, skip_special_tokens=True)


        negative_input = tokenizer.encode(f"{negative_passage} </s>", return_tensors='pt').to(device)
        negative_outputs = model.generate(input_ids=negative_input,max_length=64,do_sample=True,top_k=5,num_return_sequences=5)
        for j , no in enumerate(negative_outputs):
            dict_org['NEGATIVE_QUERY_'+str(j+1)][i] = tokenizer.decode(no, skip_special_tokens=True)
    
    return dict_org

In [14]:
df_original = pd.DataFrame.from_records(ds,columns=["ORIGINAL_QUERY","POSITIVE_PASSAGE","NEGATIVE_PASSAGE"])
df_original.insert(0, "ID", np.arange(len(df_original)))

In [15]:
df_original.head()

,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE
0,0,is a little caffeine ok during pregnancy,We dont know a lot about the effects of caffei...,It is generally safe for pregnant women to eat...
1,1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [16]:
df_queries = df_original.copy()

## Medição de performance

In [17]:
df_part = df_queries[:10]
df_part.head()

,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE
0,0,is a little caffeine ok during pregnancy,We dont know a lot about the effects of caffei...,It is generally safe for pregnant women to eat...
1,1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


### Dataframe

In [18]:
%%time
df_part_calc= df_part.apply(lambda row : passage2queryRow(row), axis = 1)

Wall time: 26.3 s


In [19]:
df_part_calc.head()

,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE
0,0,is a little caffeine ok during pregnancy,how much caffeine should i take while pregnant,can pregnant women eat chocolate
1,1,what fruit is native to australia,what is the name of the passion fruit in austr...,what is kola nut
2,2,how large is the canadian military,when was the first peacekeeping mission in canada,what is cphi
3,3,types of fruit trees,what kind of trees are there,what is kola nut
4,4,how many calories a day are lost breastfeeding,how many calories does breastfeeding burn,how much niacin should i take a day


### Listas

In [20]:
lol = df_part.values.tolist()

In [21]:
%%time
new_lol =  [ passage2queryRow(line) for line in lol]
new_lol

Wall time: 23.9 s


[[0,
  'is a little caffeine ok during pregnancy',
  'how much caffeine should i take while pregnant',
  'can pregnant women eat chocolate'],
 [1,
  'what fruit is native to australia',
  'what is the name of the passion fruit in australia',
  'what is kola nut'],
 [2,
  'how large is the canadian military',
  'when was the first peacekeeping mission in canada',
  'what is cphi'],
 [3,
  'types of fruit trees',
  'what kind of trees are there',
  'what is kola nut'],
 [4,
  'how many calories a day are lost breastfeeding',
  'how many calories does breastfeeding burn',
  'how much niacin should i take a day'],
 [5,
  'where is harrison city pa',
  'where is harrison city pa',
  'what is the closest township to forest city pa'],
 [6,
  'how many representatives does texas have',
  'how many congressional districts are there in the united states',
  'how many representatives does texas have'],
 [7,
  'major climate characteristics of tropical rainforests',
  'what is the tropical rainfor

### Dicionarios

In [46]:
dict_org = df_part.to_dict('dict')

In [29]:
%%time
dict_org = passage2queryDict(dict_org)
dict_org

Wall time: 28.2 s


{'ID': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9},
 'ORIGINAL_QUERY': {0: 'is a little caffeine ok during pregnancy',
  1: 'what fruit is native to australia',
  2: 'how large is the canadian military',
  3: 'types of fruit trees',
  4: 'how many calories a day are lost breastfeeding',
  5: 'where is harrison city pa',
  6: 'how many representatives does texas have',
  7: 'major climate characteristics of tropical rainforests',
  8: 'cancer of the pancreas symptoms',
  9: 'foods that will help lower blood sugar'},
 'POSITIVE_PASSAGE': {0: 'how much caffeine should i take while pregnant',
  1: 'what is the name of the passion fruit in australia',
  2: 'when was the first peacekeeping mission in canada',
  3: 'what kind of trees are there',
  4: 'how many calories does breastfeeding burn',
  5: 'where is harrison city pa',
  6: 'how many congressional districts are there in the united states',
  7: 'what is the tropical rainforest climate',
  8: 'pancreatic cancer sympt

In [30]:
df_part_calc = pd.DataFrame(dict_org)
df_part_calc.head()

,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE
0,0,is a little caffeine ok during pregnancy,how much caffeine should i take while pregnant,can pregnant women eat chocolate
1,1,what fruit is native to australia,what is the name of the passion fruit in austr...,what is kola nut
2,2,how large is the canadian military,when was the first peacekeeping mission in canada,what is cphi
3,3,types of fruit trees,what kind of trees are there,what is kola nut
4,4,how many calories a day are lost breastfeeding,how many calories does breastfeeding burn,how much niacin should i take a day


### Dataframe Multi Core

In [44]:
# import dask.dataframe as dd
# ddf_part = dd.from_pandas(df_part, npartitions=cpu_count())

In [ ]:
# %%time
# ddf_part_calc= ddf_part.map_partitions(lambda df_part: df_part.apply(lambda row : passage2queryRow(row), axis = 1)).compute(scheduler='processes')

### Dataframe em GPU (precisa estar em linux ou no coolab) [NÂO FUNCIONA]

**Aparentemente é complicado rodar strings em GPU**<br>

**O nvstrings surgiu como solução, mas misteriosamente ele derruba a GPU**

In [ ]:
# import cudf
# import nvstrings
# s1 = list(df_part['POSITIVE_PASSAGE'])
# s2 = list(df_part['NEGATIVE_PASSAGE'])
# df_part_gpu = cudf.from_pandas(df_part)
# s1 = nvstrings.to_device(s1)
# s2 = nvstrings.to_device(s1)

In [ ]:
# df_part_gpu = df_part_gpu.apply_rows(passage2queryGPU,
#                    incols=['POSITIVE_PASSAGE','NEGATIVE_PASSAGE'],
#                    outcols=dict(out1=np.str, out2=np.str),
#                    kwargs=dict(kwarg1=np.array(3)),
#                    chunks=16,
#                    tpb=8)

### -------------------------------------

In [57]:
df_part_new = df_part.reindex(columns = df_part.columns.tolist() + ['POSITIVE_QUERY_1','POSITIVE_QUERY_2','POSITIVE_QUERY_3','POSITIVE_QUERY_4','POSITIVE_QUERY_5','NEGATIVE_QUERY_1','NEGATIVE_QUERY_2','NEGATIVE_QUERY_3','NEGATIVE_QUERY_4','NEGATIVE_QUERY_5'])
df_part_new.head()

,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE,POSITIVE_QUERY_1,POSITIVE_QUERY_2,POSITIVE_QUERY_3,POSITIVE_QUERY_4,POSITIVE_QUERY_5,NEGATIVE_QUERY_1,NEGATIVE_QUERY_2,NEGATIVE_QUERY_3,NEGATIVE_QUERY_4,NEGATIVE_QUERY_5
0,0,is a little caffeine ok during pregnancy,We dont know a lot about the effects of caffei...,It is generally safe for pregnant women to eat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
dict_org = df_part_new.to_dict('dict')

In [61]:
%%time
dict_org = passage2queryDictMultiple(dict_org)
df_part_calc = pd.DataFrame(dict_org)
df_part_calc.head()

Wall time: 42.9 s


,ID,ORIGINAL_QUERY,POSITIVE_PASSAGE,NEGATIVE_PASSAGE,POSITIVE_QUERY_1,POSITIVE_QUERY_2,POSITIVE_QUERY_3,POSITIVE_QUERY_4,POSITIVE_QUERY_5,NEGATIVE_QUERY_1,NEGATIVE_QUERY_2,NEGATIVE_QUERY_3,NEGATIVE_QUERY_4,NEGATIVE_QUERY_5
0,0,is a little caffeine ok during pregnancy,We dont know a lot about the effects of caffei...,It is generally safe for pregnant women to eat...,how much caffeine should you have while pregnant,does coffee affect pregnancy,how much caffeine during pregnancy,how many ounces of caffeine can you drink whil...,what's a normal amount of caffeine to drink wh...,can a pregnant woman eat chocolate,is chocolate safe when pregnant,can pregnant women eat chocolate,can pregnant women eat chocolate,is chocolate safe while pregnant
1,1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ...",what is Passiflora herbertiana,what is passion fruit,what is a passion fruit,what is a passion fruit in australia,why do some passionfruit grow wild,what is the difference between kola nut and ko...,what is kola nut,what is kola nut,where are kola nuts found,where is kola nut found
2,2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...,when was peacekeeping started canadian,when did the peacekeeping missions start,what day is peacekeeping in canada,where did peacekeeping begin,when did peacekeeping began?,who is cmf cphi,what is cphi,who is cphi,what is cphi canada,what is the cphi program
3,3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ...",different varieties of cherries,why do i grow cherry trees,what kind of fruit do cherry trees grow,where does the cherry tree grow,where are cherries found in the world,where is kola nut from,what is kolanut,where is kola nut from,what is kola nut,what is kola nut
4,4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ...",how many calories does breast feeding burn,how many calories do i burn breastfeeding,how many calories burned breastfeeding,calories burned breastfeeding a baby,calories burned breastfeeding,what is the recommended niacin level for women,how much niacin should you take,how much niacin do i need,how much niacin does a woman need daily,how many niacin should i take per day


In [ ]:
df_queries_new = df_queries.reindex(columns = df_queries.columns.tolist() + ['POSITIVE_QUERY_1','POSITIVE_QUERY_2','POSITIVE_QUERY_3','POSITIVE_QUERY_4','POSITIVE_QUERY_5','NEGATIVE_QUERY_1','NEGATIVE_QUERY_2','NEGATIVE_QUERY_3','NEGATIVE_QUERY_4','NEGATIVE_QUERY_5'])
df_queries_new.head()
dict_queries = df_queries_new.to_dict('dict')
dict_queries = passage2queryDictMultiple(dict_queries)
df_queries = pd.DataFrame(dict_queries)
df_queries.head()

## Transformação dos documentos em queries

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='checkup_dataframe.csv',
                       encoding = 'cp1252')  
df_queries.to_csv('checkup_queries_dataframe.zip', index=False,
          compression=compression_opts)  